In [20]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)  #toyota
# data = web.DataReader('1321.T', 'yahoo', start, end)  #(NEXT FUNDS)日経225連動型上場投信
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = web.DataReader('2558.T', 'yahoo', start, end)  #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)  #MAXIS ナスダック100上場投信
# data = web.DataReader('2559.T', 'yahoo', start, end)    #MAXIS 全世界株式(オール・カントリー)上場投信

# data = web.DataReader('BTC-JPY', 'yahoo', start, end)   #Bitcoin
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               High      Low     Open    Close    Volume  Adj Close
Date                                                               
2020-01-07   9917.0   9917.0   9917.0   9917.0       0.0     9917.0
2020-01-08   9917.0   9917.0   9917.0   9917.0       0.0     9917.0
2020-01-09  10097.0  10097.0  10097.0  10097.0       0.0    10097.0
2020-01-10  10320.0  10280.0  10300.0  10320.0    2502.0    10320.0
2020-01-14  10400.0  10350.0  10390.0  10370.0    5739.0    10370.0
...             ...      ...      ...      ...       ...        ...
2022-09-13  16910.0  16840.0  16885.0  16890.0   48912.0    16890.0
2022-09-14  16420.0  16275.0  16370.0  16280.0  118132.0    16280.0
2022-09-15  16330.0  16245.0  16265.0  16320.0   35042.0    16320.0
2022-09-16  16015.0  15945.0  16000.0  16005.0   42610.0    16005.0
2022-09-20  16140.0  16065.0  16130.0  16140.0   19472.0    16140.0

[661 rows x 6 columns]


**Methods**

In [21]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [22]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**MACD CROSS**

In [23]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   54.311649
Equity Final [$]                1193535.80925
Equity Peak [$]                  1287505.4235
Return [%]                          19.353581
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    6.977585
Volatility (Ann.) [%]               14.792564
Sharpe Ratio                         0.471695
Sortino Ratio                        0.739941
Calmar Ratio                         0.505519
Max. Drawdown [%]                   -13.80281
Avg. Drawdown [%]                   -4.009449
Max. Drawdown Duration      511 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                   26
Win Rate [%]                        46.153846
Best Trade [%]                      12.169696
Worst Trade [%]                     -6.068985
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,127,55,85,7818.51100,8770.0,120839.10300,0.121697,2020-03-30,2020-05-15,46 days
1,123,88,106,9064.64900,9190.0,15418.17300,0.013829,2020-05-20,2020-06-15,26 days
2,115,122,136,9818.36150,9690.0,-14761.57250,-0.013074,2020-07-07,2020-07-29,22 days
3,111,142,164,10069.59900,10460.0,43334.51100,0.038770,2020-08-06,2020-09-08,33 days
4,114,182,194,10190.19300,10410.0,25057.99800,0.021570,2020-10-07,2020-10-23,16 days
5,110,205,227,10752.96500,10920.0,18373.85000,0.015534,2020-11-10,2020-12-11,31 days
6,110,242,243,10974.05400,10920.0,-5945.94000,-0.004926,2021-01-05,2021-01-06,1 days
7,105,246,259,11386.08350,11250.0,-14288.76750,-0.011952,2021-01-12,2021-01-29,17 days
8,101,264,276,11737.81600,11930.0,19410.58400,0.016373,2021-02-05,2021-02-25,20 days
9,99,286,295,12190.04350,12160.0,-2974.30650,-0.002465,2021-03-11,2021-03-24,13 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   53.706505
Equity Final [$]                   1235554.19
Equity Peak [$]                 1360711.58025
Return [%]                          23.555419
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    8.398043
Volatility (Ann.) [%]               15.677507
Sharpe Ratio                         0.535675
Sortino Ratio                        0.857941
Calmar Ratio                         0.639982
Max. Drawdown [%]                  -13.122301
Avg. Drawdown [%]                   -3.429438
Max. Drawdown Duration      424 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                   22
Win Rate [%]                        45.454545
Best Trade [%]                      18.508859
Worst Trade [%]                     -5.708083
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,125,57,107,7999.40200,9480.0,185074.75000,0.185089,2020-04-01,2020-06-16,76 days
1,121,123,126,9727.91600,9740.0,1462.16400,0.001242,2020-07-08,2020-07-13,5 days
2,121,127,137,9748.01500,9770.0,2660.18500,0.002255,2020-07-14,2020-07-30,16 days
3,118,142,164,10069.59900,10460.0,46067.31800,0.038770,2020-08-06,2020-09-08,33 days
4,116,184,195,10562.02450,10350.0,-24594.84200,-0.020074,2020-10-09,2020-10-26,17 days
5,112,205,228,10752.96500,10920.0,18707.92000,0.015534,2020-11-10,2020-12-14,34 days
6,107,246,259,11386.08350,11250.0,-14560.93450,-0.011952,2021-01-12,2021-01-29,17 days
7,103,264,276,11737.81600,11930.0,19794.95200,0.016373,2021-02-05,2021-02-25,20 days
8,105,277,278,11707.66750,11690.0,-1855.08750,-0.001509,2021-02-26,2021-03-01,3 days
9,99,287,296,12330.73650,12150.0,-17892.91350,-0.014657,2021-03-12,2021-03-25,13 days


<Strategy MACDCross(MACDshort=15,MACDlong=40,MACDsignal=10)>

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   54.311649
Equity Final [$]                1193535.80925
Equity Peak [$]                  1287505.4235
Return [%]                          19.353581
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    6.977585
Volatility (Ann.) [%]               14.792564
Sharpe Ratio                         0.471695
Sortino Ratio                        0.739941
Calmar Ratio                         0.505519
Max. Drawdown [%]                   -13.80281
Avg. Drawdown [%]                   -4.009449
Max. Drawdown Duration      511 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                   26
Win Rate [%]                        46.153846
Best Trade [%]                      12.169696
Worst Trade [%]                     -6.068985
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,127,55,85,7818.51100,8770.0,120839.10300,0.121697,2020-03-30,2020-05-15,46 days
1,123,88,106,9064.64900,9190.0,15418.17300,0.013829,2020-05-20,2020-06-15,26 days
2,115,122,136,9818.36150,9690.0,-14761.57250,-0.013074,2020-07-07,2020-07-29,22 days
3,111,142,164,10069.59900,10460.0,43334.51100,0.038770,2020-08-06,2020-09-08,33 days
4,114,182,194,10190.19300,10410.0,25057.99800,0.021570,2020-10-07,2020-10-23,16 days
5,110,205,227,10752.96500,10920.0,18373.85000,0.015534,2020-11-10,2020-12-11,31 days
6,110,242,243,10974.05400,10920.0,-5945.94000,-0.004926,2021-01-05,2021-01-06,1 days
7,105,246,259,11386.08350,11250.0,-14288.76750,-0.011952,2021-01-12,2021-01-29,17 days
8,101,264,276,11737.81600,11930.0,19410.58400,0.016373,2021-02-05,2021-02-25,20 days
9,99,286,295,12190.04350,12160.0,-2974.30650,-0.002465,2021-03-11,2021-03-24,13 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   53.706505
Equity Final [$]                   1235554.19
Equity Peak [$]                 1360711.58025
Return [%]                          23.555419
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    8.398043
Volatility (Ann.) [%]               15.677507
Sharpe Ratio                         0.535675
Sortino Ratio                        0.857941
Calmar Ratio                         0.639982
Max. Drawdown [%]                  -13.122301
Avg. Drawdown [%]                   -3.429438
Max. Drawdown Duration      424 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                   22
Win Rate [%]                        45.454545
Best Trade [%]                      18.508859
Worst Trade [%]                     -5.708083
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,125,57,107,7999.40200,9480.0,185074.75000,0.185089,2020-04-01,2020-06-16,76 days
1,121,123,126,9727.91600,9740.0,1462.16400,0.001242,2020-07-08,2020-07-13,5 days
2,121,127,137,9748.01500,9770.0,2660.18500,0.002255,2020-07-14,2020-07-30,16 days
3,118,142,164,10069.59900,10460.0,46067.31800,0.038770,2020-08-06,2020-09-08,33 days
4,116,184,195,10562.02450,10350.0,-24594.84200,-0.020074,2020-10-09,2020-10-26,17 days
5,112,205,228,10752.96500,10920.0,18707.92000,0.015534,2020-11-10,2020-12-14,34 days
6,107,246,259,11386.08350,11250.0,-14560.93450,-0.011952,2021-01-12,2021-01-29,17 days
7,103,264,276,11737.81600,11930.0,19794.95200,0.016373,2021-02-05,2021-02-25,20 days
8,105,277,278,11707.66750,11690.0,-1855.08750,-0.001509,2021-02-26,2021-03-01,3 days
9,99,287,296,12330.73650,12150.0,-17892.91350,-0.014657,2021-03-12,2021-03-25,13 days


<Strategy MACDCross_WithShortPosition(MACDshort=15,MACDlong=40,MACDsignal=10)>

**EMA CROSS**

In [24]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "EMAshort": range(10, 100, 10),\
    "EMAlong": range(10, 200, 5),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint":  lambda x: (x.EMAlong > x.EMAshort),\
    "max_tries":  1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True) #自動でポジションをクローズ(オープン)
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   72.163389
Equity Final [$]                  1175220.152
Equity Peak [$]                 1299375.03075
Return [%]                          17.522015
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    6.348726
Volatility (Ann.) [%]               16.801542
Sharpe Ratio                         0.377866
Sortino Ratio                        0.565736
Calmar Ratio                         0.435642
Max. Drawdown [%]                  -14.573259
Avg. Drawdown [%]                   -2.904861
Max. Drawdown Duration      163 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   30
Win Rate [%]                        46.666667
Best Trade [%]                       7.958852
Worst Trade [%]                     -6.281262
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,94,22,34,10562.02450,9980.0,-54710.30300,-0.055105,2020-02-07,2020-02-27,20 days
1,118,57,59,7999.40200,7950.0,-5829.43600,-0.006176,2020-04-01,2020-04-03,2 days
2,112,61,85,8341.08500,8770.0,48038.48000,0.051422,2020-04-07,2020-05-15,38 days
3,108,87,107,9104.84700,9480.0,40516.52400,0.041204,2020-05-19,2020-06-16,28 days
4,106,120,137,9657.56950,9770.0,11917.63300,0.011642,2020-07-03,2020-07-30,27 days
5,103,140,166,10019.35150,10370.0,36116.79550,0.034997,2020-08-04,2020-09-10,37 days
6,105,182,194,10190.19300,10410.0,23079.73500,0.021570,2020-10-07,2020-10-23,16 days
7,104,204,230,10541.92550,10930.0,40359.74800,0.036812,2020-11-09,2020-12-16,37 days
8,103,232,236,11024.30150,10920.0,-10743.05450,-0.009461,2020-12-18,2020-12-24,6 days
9,102,237,244,10994.15300,11070.0,7736.39400,0.006899,2020-12-25,2021-01-07,13 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   73.524962
Equity Final [$]                  1211682.991
Equity Peak [$]                   1369393.173
Return [%]                          21.168299
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    7.594797
Volatility (Ann.) [%]               17.539029
Sharpe Ratio                         0.433023
Sortino Ratio                        0.655701
Calmar Ratio                         0.532523
Max. Drawdown [%]                  -14.261918
Avg. Drawdown [%]                   -2.424396
Max. Drawdown Duration      299 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   13
Win Rate [%]                        46.153846
Best Trade [%]                      30.754634
Worst Trade [%]                    -11.569984
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,94,22,35,10562.02450,9340.0,-114870.30300,-0.115700,2020-02-07,2020-02-28,21 days
1,103,64,117,8592.32250,9390.0,82160.78250,0.092836,2020-04-10,2020-06-30,81 days
2,100,120,168,9657.56950,10200.0,54243.05000,0.056166,2020-07-03,2020-09-14,73 days
3,96,184,198,10562.02450,9840.0,-69314.35200,-0.068360,2020-10-09,2020-10-29,20 days
4,88,205,415,10752.96500,14060.0,291019.08000,0.307546,2020-11-10,2021-09-17,311 days
5,87,431,466,14199.94350,14875.0,58729.91550,0.047539,2021-10-13,2021-12-03,51 days
6,85,473,478,15290.31425,14955.0,-28501.71125,-0.021930,2021-12-14,2021-12-21,7 days
7,81,481,495,15551.60125,15330.0,-17949.70125,-0.014249,2021-12-24,2022-01-18,25 days
8,85,513,514,14752.66600,14625.0,-10851.61000,-0.008654,2022-02-14,2022-02-15,1 days
9,82,536,564,15018.97775,15565.0,44773.82450,0.036355,2022-03-18,2022-04-28,41 days


<Strategy EmaCrossStrategy(EMAshort=10,EMAlong=15)>

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   72.919818
Equity Final [$]                 1274669.5815
Equity Peak [$]                  1409988.0555
Return [%]                          27.466958
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    9.693741
Volatility (Ann.) [%]               17.493383
Sharpe Ratio                         0.554138
Sortino Ratio                        0.884499
Calmar Ratio                         0.664821
Max. Drawdown [%]                   -14.58098
Avg. Drawdown [%]                   -2.816741
Max. Drawdown Duration      163 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   30
Win Rate [%]                             50.0
Best Trade [%]                       7.958852
Worst Trade [%]                     -6.281262
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-97,17,34,10229.11400,9980.0,24164.05800,0.024353,2020-01-31,2020-02-27,27 days
1,128,57,59,7999.40200,7950.0,-6323.45600,-0.006176,2020-04-01,2020-04-03,2 days
2,122,61,85,8341.08500,8770.0,52327.63000,0.051422,2020-04-07,2020-05-15,38 days
3,117,87,107,9104.84700,9480.0,43892.90100,0.041204,2020-05-19,2020-06-16,28 days
4,115,120,137,9657.56950,9770.0,12929.50750,0.011642,2020-07-03,2020-07-30,27 days
5,112,140,166,10019.35150,10370.0,39272.63200,0.034997,2020-08-04,2020-09-10,37 days
6,114,182,194,10190.19300,10410.0,25057.99800,0.021570,2020-10-07,2020-10-23,16 days
7,113,204,230,10541.92550,10930.0,43852.41850,0.036812,2020-11-09,2020-12-16,37 days
8,112,232,236,11024.30150,10920.0,-11681.76800,-0.009461,2020-12-18,2020-12-24,6 days
9,111,237,244,10994.15300,11070.0,8419.01700,0.006899,2020-12-25,2021-01-07,13 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   74.281392
Equity Final [$]                1487478.22775
Equity Peak [$]                  1680044.8785
Return [%]                          48.747823
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   16.344315
Volatility (Ann.) [%]               19.029272
Sharpe Ratio                         0.858904
Sortino Ratio                        1.499383
Calmar Ratio                         1.186003
Max. Drawdown [%]                  -13.781002
Avg. Drawdown [%]                   -2.113469
Max. Drawdown Duration      299 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   13
Win Rate [%]                        53.846154
Best Trade [%]                      30.754634
Worst Trade [%]                     -7.234517
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-97,17,35,10229.11400,9340.0,86244.05800,0.086920,2020-01-31,2020-02-28,28 days
1,126,64,117,8592.32250,9390.0,100507.36500,0.092836,2020-04-10,2020-06-30,81 days
2,122,120,168,9657.56950,10200.0,66176.52100,0.056166,2020-07-03,2020-09-14,73 days
3,118,184,198,10562.02450,9840.0,-85198.89100,-0.068360,2020-10-09,2020-10-29,20 days
4,108,205,415,10752.96500,14060.0,357159.78000,0.307546,2020-11-10,2021-09-17,311 days
5,107,431,466,14199.94350,14875.0,72231.04550,0.047539,2021-10-13,2021-12-03,51 days
6,104,473,478,15290.31425,14955.0,-34872.68200,-0.021930,2021-12-14,2021-12-21,7 days
7,100,481,495,15551.60125,15330.0,-22160.12500,-0.014249,2021-12-24,2022-01-18,25 days
8,104,513,514,14752.66600,14625.0,-13277.26400,-0.008654,2022-02-14,2022-02-15,1 days
9,101,536,564,15018.97775,15565.0,55148.24725,0.036355,2022-03-18,2022-04-28,41 days


<Strategy EmaCrossStrategy_WithShortPosition(EMAshort=10,EMAlong=15)>

**SMA**

In [25]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "SMA_short": range(10, 100, 5),\
    "SMA_long": range(10, 200, 5),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000,\
    "constraint": lambda x: x.SMA_short < x.SMA_long}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   68.532526
Equity Final [$]                 1231016.0095
Equity Peak [$]                  1413338.1555
Return [%]                          23.101601
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    8.246081
Volatility (Ann.) [%]               16.804673
Sharpe Ratio                         0.490702
Sortino Ratio                        0.761368
Calmar Ratio                         0.402408
Max. Drawdown [%]                   -20.49186
Avg. Drawdown [%]                   -2.720002
Max. Drawdown Duration      299 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                      20.417757
Worst Trade [%]                      -7.84595
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,114,67,118,8743.06500,9520.0,88570.59000,0.088863,2020-04-15,2020-07-01,77 days
1,111,125,172,9727.91600,10070.0,37971.32400,0.035165,2020-07-10,2020-09-18,70 days
2,106,188,200,10582.12350,9780.0,-85025.09100,-0.075800,2020-10-15,2020-11-02,18 days
3,95,209,335,10903.70750,13130.0,211497.78750,0.204178,2020-11-16,2021-05-25,190 days
4,94,337,418,13215.09250,14100.0,83181.30500,0.066962,2021-05-27,2021-09-24,120 days
5,90,435,469,14792.86400,15340.0,49242.24000,0.036986,2021-10-19,2021-12-08,50 days
6,87,484,498,15843.03675,14600.0,-108144.19725,-0.078460,2021-12-29,2022-01-21,23 days
7,81,539,567,15591.79925,15370.0,-17965.73925,-0.014225,2022-03-24,2022-05-09,46 days
8,78,590,599,15953.58125,14710.0,-96999.33750,-0.077950,2022-06-09,2022-06-22,13 days
9,76,616,653,15265.19050,16490.0,93085.52200,0.080235,2022-07-15,2022-09-08,55 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   72.768533
Equity Final [$]                  1009632.469
Equity Peak [$]                  1255242.0265
Return [%]                           0.963247
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                     0.36614
Volatility (Ann.) [%]               17.377558
Sharpe Ratio                          0.02107
Sortino Ratio                        0.029149
Calmar Ratio                          0.01559
Max. Drawdown [%]                  -23.485443
Avg. Drawdown [%]                    -4.20051
Max. Drawdown Duration      299 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                   26
Win Rate [%]                        65.384615
Best Trade [%]                       12.64993
Worst Trade [%]                    -13.924174
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,94,26,36,10572.07400,9100.0,-138374.95600,-0.139242,2020-02-14,2020-03-02,17 days
1,107,59,81,7989.35250,9000.0,108139.28250,0.126499,2020-04-03,2020-05-11,38 days
2,107,82,112,9014.40150,9540.0,56239.03950,0.058307,2020-05-12,2020-06-23,42 days
3,107,119,122,9577.17350,9770.0,20632.43550,0.020134,2020-07-02,2020-07-07,5 days
4,107,123,143,9727.91600,10090.0,38742.98800,0.037221,2020-07-08,2020-08-07,30 days
5,106,145,169,10220.34150,10250.0,3143.80100,0.002902,2020-08-12,2020-09-15,34 days
6,102,185,197,10582.12350,10110.0,-48156.59700,-0.044615,2020-10-12,2020-10-28,16 days
7,97,208,234,10722.81650,10910.0,18156.79950,0.017457,2020-11-13,2020-12-22,39 days
8,95,240,281,11094.64800,11650.0,52758.44000,0.050056,2020-12-30,2021-03-04,64 days
9,91,286,301,12190.04350,12640.0,40946.04150,0.036912,2021-03-11,2021-04-01,21 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-110,36,118,9054.95500,9520.0,-51154.95000,-0.051358,2020-03-02,2020-07-01,121 days
1,97,125,172,9727.91600,10070.0,33182.14800,0.035165,2020-07-10,2020-09-18,70 days
2,92,188,200,10582.12350,9780.0,-73795.36200,-0.075800,2020-10-15,2020-11-02,18 days
3,83,209,335,10903.70750,13130.0,184782.27750,0.204178,2020-11-16,2021-05-25,190 days
4,82,337,418,13215.09250,14100.0,72562.41500,0.066962,2021-05-27,2021-09-24,120 days
5,78,435,469,14792.86400,15340.0,42676.60800,0.036986,2021-10-19,2021-12-08,50 days
6,76,484,498,15843.03675,14600.0,-94470.79300,-0.078460,2021-12-29,2022-01-21,23 days
7,71,539,567,15591.79925,15370.0,-15747.74675,-0.014225,2022-03-24,2022-05-09,46 days
8,68,590,599,15953.58125,14710.0,-84563.52500,-0.077950,2022-06-09,2022-06-22,13 days
9,66,616,653,15265.19050,16490.0,80837.42700,0.080235,2022-07-15,2022-09-08,55 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                    73.97882
Equity Final [$]                 1288992.1115
Equity Peak [$]                   1601146.172
Return [%]                          28.899211
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   10.162014
Volatility (Ann.) [%]               19.155498
Sharpe Ratio                         0.530501
Sortino Ratio                        0.879829
Calmar Ratio                         0.433521
Max. Drawdown [%]                  -23.440626
Avg. Drawdown [%]                   -3.297838
Max. Drawdown Duration      299 days 00:00:00
Avg. Drawdown Duration       32 days 00:00:00
# Trades                                   26
Win Rate [%]                        69.230769
Best Trade [%]                       12.64993
Worst Trade [%]                    -11.305131
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-99,18,36,10099.75750,9100.0,98975.9925,0.098988,2020-02-03,2020-03-02,28 days
1,137,59,81,7989.35250,9000.0,138458.7075,0.126499,2020-04-03,2020-05-11,38 days
2,137,82,112,9014.40150,9540.0,72006.9945,0.058307,2020-05-12,2020-06-23,42 days
3,136,119,122,9577.17350,9770.0,26224.4040,0.020134,2020-07-02,2020-07-07,5 days
4,137,123,143,9727.91600,10090.0,49605.5080,0.037221,2020-07-08,2020-08-07,30 days
5,135,145,169,10220.34150,10250.0,4003.8975,0.002902,2020-08-12,2020-09-15,34 days
6,131,185,197,10582.12350,10110.0,-61848.1785,-0.044615,2020-10-12,2020-10-28,16 days
7,123,208,234,10722.81650,10910.0,23023.5705,0.017457,2020-11-13,2020-12-22,39 days
8,121,240,281,11094.64800,11650.0,67197.5920,0.050056,2020-12-30,2021-03-04,64 days
9,116,286,301,12190.04350,12640.0,52194.9540,0.036912,2021-03-11,2021-04-01,21 days


<Strategy SmaCross_WithShortPosition(SMA_short=10,SMA_long=15)>

**ボリンジャーバンド**

In [26]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "n": range(10, 40, 5),\
    "nu": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "nd": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   36.611195
Equity Final [$]                 1344213.8745
Equity Peak [$]                  1355688.8745
Return [%]                          34.421387
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   11.937951
Volatility (Ann.) [%]               19.562633
Sharpe Ratio                         0.610243
Sortino Ratio                        0.973418
Calmar Ratio                          0.39544
Max. Drawdown [%]                   -30.18907
Avg. Drawdown [%]                   -4.524049
Max. Drawdown Duration      269 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                    7
Win Rate [%]                        85.714286
Best Trade [%]                      12.327415
Worst Trade [%]                     -10.06442
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,99,34,88,10029.40100,9020.0,-99930.69900,-0.100644,2020-02-27,2020-05-20,83 days
1,91,175,187,9888.70800,10630.0,67457.57200,0.074963,2020-09-25,2020-10-14,19 days
2,98,200,240,9828.41100,11040.0,118735.72200,0.123274,2020-11-02,2020-12-30,58 days
3,79,417,434,13677.36950,14720.0,82367.80950,0.076230,2021-09-22,2021-10-18,26 days
4,78,466,484,14948.63125,15765.0,63676.76250,0.054612,2021-12-03,2021-12-29,26 days
5,85,499,538,14496.40375,15740.0,105705.68125,0.085787,2022-01-24,2022-03-23,58 days
6,85,564,619,15642.04675,15715.0,6201.02625,0.004664,2022-04-28,2022-07-21,84 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 726 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   69.742814
Equity Final [$]                  1408198.338
Equity Peak [$]                 1497384.03675
Return [%]                          40.819834
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   13.940122
Volatility (Ann.) [%]               22.400727
Sharpe Ratio                         0.622307
Sortino Ratio                        0.999106
Calmar Ratio                         0.459453
Max. Drawdown [%]                  -30.340689
Avg. Drawdown [%]                     -2.9688
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                      18.435964
Worst Trade [%]                     -3.284354
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,97,14,31,10250.49000,10740.0,47482.47000,0.047755,2020-01-28,2020-02-21,24 days
1,104,34,100,10029.40100,9700.0,-34257.70400,-0.032844,2020-02-27,2020-06-05,99 days
2,106,107,133,9526.92600,9960.0,45905.84400,0.045458,2020-06-16,2020-07-22,36 days
3,101,166,240,10421.33150,11040.0,62485.51850,0.059366,2020-09-10,2020-12-30,111 days
4,99,259,350,11305.68750,13390.0,206346.93750,0.184360,2021-01-29,2021-06-15,137 days
5,99,355,388,13406.03300,14040.0,62762.73300,0.047290,2021-06-22,2021-08-11,50 days
6,97,411,434,14199.94350,14720.0,50445.48050,0.036624,2021-09-13,2021-10-18,35 days
7,93,463,536,15340.56175,14945.0,-36787.24275,-0.025785,2021-11-30,2022-03-18,108 days
8,89,564,603,15642.04675,15175.0,-41567.16075,-0.029858,2022-04-28,2022-06-28,61 days
9,84,646,655,16189.74450,16730.0,45381.46200,0.033370,2022-08-30,2022-09-12,13 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-94,28,31,10557.48050,10740.0,-17156.83300,-0.017288,2020-02-18,2020-02-21,3 days
1,97,34,88,10029.40100,9020.0,-97911.89700,-0.100644,2020-02-27,2020-05-20,83 days
2,-96,93,95,9164.41050,9310.0,-13976.59200,-0.015886,2020-05-27,2020-05-29,2 days
3,-89,99,100,9691.78700,9700.0,-730.95700,-0.000847,2020-06-04,2020-06-05,1 days
4,-87,101,102,9970.40100,9970.0,34.88700,0.000040,2020-06-08,2020-06-09,1 days
5,-88,103,133,9831.09400,9960.0,-11343.72800,-0.013112,2020-06-10,2020-07-22,42 days
6,-84,145,146,10119.65850,10310.0,-15988.68600,-0.018809,2020-08-12,2020-08-13,1 days
7,-82,147,148,10278.86650,10290.0,-912.94700,-0.001083,2020-08-14,2020-08-17,3 days
8,-81,162,187,10378.37150,10630.0,-20381.90850,-0.024245,2020-09-04,2020-10-14,40 days
9,83,200,240,9828.41100,11040.0,100561.88700,0.123274,2020-11-02,2020-12-30,58 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 726 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   80.786687
Equity Final [$]                   719745.502
Equity Peak [$]                   1107130.913
Return [%]                          -28.02545
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                  -11.783296
Volatility (Ann.) [%]                18.54373
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.990028
Avg. Drawdown [%]                  -11.495523
Max. Drawdown Duration      827 days 00:00:00
Avg. Drawdown Duration      160 days 00:00:00
# Trades                                   14
Win Rate [%]                        28.571429
Best Trade [%]                       5.936559
Worst Trade [%]                    -18.769728
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,97,14,31,10250.49000,10740.0,47482.47000,0.047755,2020-01-28,2020-02-21,24 days
1,104,34,100,10029.40100,9700.0,-34257.70400,-0.032844,2020-02-27,2020-06-05,99 days
2,-102,102,133,9920.64850,9960.0,-4013.85300,-0.003967,2020-06-09,2020-07-22,43 days
3,-96,155,156,10448.02500,10530.0,-7869.60000,-0.007846,2020-08-26,2020-08-27,1 days
4,96,166,240,10421.33150,11040.0,59392.17600,0.059366,2020-09-10,2020-12-30,111 days
5,-94,246,350,11273.91650,13390.0,-198911.84900,-0.187697,2021-01-12,2021-06-15,154 days
6,-64,351,388,13313.76900,14040.0,-46478.78400,-0.054547,2021-06-16,2021-08-11,56 days
7,57,411,434,14199.94350,14720.0,29643.22050,0.036624,2021-09-13,2021-10-18,35 days
8,-56,445,447,15035.20550,15320.0,-15948.49200,-0.018942,2021-11-02,2021-11-05,3 days
9,-54,456,536,15333.72050,14945.0,20990.90700,0.025351,2021-11-18,2022-03-18,120 days


<Strategy BBsigma_WithShortPosition(n=10,nu=2.0,nd=2.0)>

**RSI**

In [27]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   15.582451
Equity Final [$]                 1194792.3035
Equity Peak [$]                  1194792.3035
Return [%]                           19.47923
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    7.020507
Volatility (Ann.) [%]               14.816098
Sharpe Ratio                         0.473843
Sortino Ratio                        0.736414
Calmar Ratio                         0.263822
Max. Drawdown [%]                  -26.610797
Avg. Drawdown [%]                   -9.688509
Max. Drawdown Duration       90 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      10.643281
Worst Trade [%]                      8.141489
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,107,38,101,9265.6390,10020.0,80716.6270,0.081415,2020-03-04,2020-06-08,96 days
1,73,504,542,14682.3195,16245.0,114075.6765,0.106433,2022-01-31,2022-03-29,57 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                    1.512859
Equity Final [$]                 1059872.8165
Equity Peak [$]                  1059872.8165
Return [%]                           5.987282
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    2.241628
Volatility (Ann.) [%]                5.382888
Sharpe Ratio                         0.416436
Sortino Ratio                        0.760115
Calmar Ratio                         0.446437
Max. Drawdown [%]                   -5.021153
Avg. Drawdown [%]                   -2.469337
Max. Drawdown Duration        7 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       6.022046
Worst Trade [%]                      6.022046
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,127,52,61,7828.5605,8300.0,59872.8165,0.06022,2020-03-25,2020-04-07,13 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-99,18,21,10099.75750,10400.0,-29724.00750,-0.029728,2020-02-03,2020-02-06,3 days
1,103,35,78,9386.23300,8850.0,-55231.99900,-0.057130,2020-02-28,2020-05-01,63 days
2,-99,93,133,9164.41050,9960.0,-78763.36050,-0.086813,2020-05-27,2020-07-22,56 days
3,-83,143,153,10040.05450,10290.0,-20745.47650,-0.024895,2020-08-07,2020-08-24,17 days
4,82,175,186,9888.70800,10690.0,65705.94400,0.081031,2020-09-25,2020-10-13,18 days
5,89,199,207,9898.75750,10800.0,80210.58250,0.091046,2020-10-30,2020-11-12,13 days
6,-88,210,215,10816.19350,10920.0,-9134.97200,-0.009597,2020-11-17,2020-11-25,8 days
7,-87,220,239,10905.74800,11080.0,-15159.92400,-0.015978,2020-12-02,2020-12-29,27 days
8,-83,246,252,11273.91650,11310.0,-2994.93050,-0.003201,2021-01-12,2021-01-20,8 days
9,-81,263,277,11482.87700,11650.0,-13536.96300,-0.014554,2021-02-04,2021-02-26,22 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   43.570348
Equity Final [$]                  798388.6645
Equity Peak [$]                  1318059.6535
Return [%]                         -20.161134
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   -8.225897
Volatility (Ann.) [%]               14.180365
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -39.578329
Avg. Drawdown [%]                    -9.85936
Max. Drawdown Duration      911 days 00:00:00
Avg. Drawdown Duration      189 days 00:00:00
# Trades                                   21
Win Rate [%]                        19.047619
Best Trade [%]                      19.095156
Worst Trade [%]                     -9.509153
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-97,20,61,10258.96550,8300.0,190019.65350,0.190952,2020-02-05,2020-04-07,62 days
1,-135,86,108,8766.39050,9600.0,-112537.28250,-0.095092,2020-05-18,2020-06-17,30 days
2,-114,113,119,9443.02450,9530.0,-9915.20700,-0.009211,2020-06-24,2020-07-02,8 days
3,-109,138,140,9751.49000,9970.0,-23817.59000,-0.022408,2020-07-31,2020-08-04,4 days
4,-103,165,178,10040.05450,10130.0,-9264.38650,-0.008959,2020-09-09,2020-09-30,21 days
5,-101,181,202,10239.06450,10290.0,-5144.48550,-0.004975,2020-10-06,2020-11-05,30 days
6,-94,231,237,10865.94600,10940.0,-6961.07600,-0.006815,2020-12-17,2020-12-25,8 days
7,-91,245,262,11234.11450,11510.0,-25105.58050,-0.024558,2021-01-08,2021-02-03,26 days
8,-83,279,284,11880.89700,11960.0,-6565.54900,-0.006658,2021-03-02,2021-03-09,7 days
9,-76,329,334,13035.15500,12980.0,4191.78000,0.004231,2021-05-17,2021-05-24,7 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=50,lower_bound=10,rsi_window=10)>

**RSI50エントリー**

In [28]:
import BacktestingStrategies.Strategy_RsiOscillator as btstrsi50
importlib.reload(btstrsi50)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstrsi50.RsiOscillator_Entry50, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstrsi50.RsiOscillator_Entry50_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   63.993949
Equity Final [$]                  1516974.166
Equity Peak [$]                  1598876.5725
Return [%]                          51.697417
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   17.218517
Volatility (Ann.) [%]               17.031443
Sharpe Ratio                         1.010984
Sortino Ratio                         1.74622
Calmar Ratio                         0.976573
Max. Drawdown [%]                  -17.631577
Avg. Drawdown [%]                   -2.443639
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   17
Win Rate [%]                        52.941176
Best Trade [%]                      18.981448
Worst Trade [%]                     -6.937171
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,118,63,101,8421.48100,10020.0,188625.2420,0.189814,2020-04-09,2020-06-08,60 days
1,123,108,157,9647.52000,10650.0,123305.0400,0.103911,2020-06-17,2020-08-28,72 days
2,128,167,175,10230.39100,9840.0,-49970.0480,-0.038160,2020-09-11,2020-09-25,14 days
3,123,178,199,10180.14350,9850.0,-40607.6505,-0.032430,2020-09-30,2020-10-30,30 days
4,116,203,270,10471.57900,11920.0,168016.8360,0.138319,2020-11-06,2021-02-16,102 days
5,115,279,302,11999.10300,12770.0,88653.1550,0.064246,2021-03-02,2021-04-02,31 days
6,112,329,364,13164.84500,13840.0,75617.3600,0.051285,2021-05-17,2021-07-05,49 days
7,110,377,417,14019.05250,13610.0,-44995.7750,-0.029178,2021-07-26,2021-09-22,58 days
8,105,419,424,14250.19100,13900.0,-36770.0550,-0.024574,2021-09-27,2021-10-04,7 days
9,103,428,437,14189.89400,14880.0,71080.9180,0.048634,2021-10-08,2021-10-21,13 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   50.226929
Equity Final [$]                1338809.36625
Equity Peak [$]                 1401129.36625
Return [%]                          33.880937
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                   11.766158
Volatility (Ann.) [%]               15.732098
Sharpe Ratio                         0.747908
Sortino Ratio                        1.293395
Calmar Ratio                         0.835501
Max. Drawdown [%]                  -14.082762
Avg. Drawdown [%]                   -3.486575
Max. Drawdown Duration      312 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                   29
Win Rate [%]                        48.275862
Best Trade [%]                      18.981448
Worst Trade [%]                     -6.937171
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-100,34,36,9930.59900,9100.0,83059.90000,0.083640,2020-02-27,2020-03-02,4 days
1,128,63,101,8421.48100,10020.0,204610.43200,0.189814,2020-04-09,2020-06-08,60 days
2,-136,107,108,9433.07400,9600.0,-22701.93600,-0.017696,2020-06-16,2020-06-17,1 days
3,-134,115,116,9373.37100,9200.0,23231.71400,0.018496,2020-06-26,2020-06-29,3 days
4,-137,117,118,9343.51950,9520.0,-24177.82850,-0.018888,2020-06-30,2020-07-01,1 days
5,-130,137,138,9721.63850,9800.0,-10186.99500,-0.008061,2020-07-30,2020-07-31,1 days
6,-121,166,167,10318.66850,10180.0,16778.88850,0.013439,2020-09-10,2020-09-11,1 days
7,-125,168,169,10149.51000,10250.0,-12561.25000,-0.009901,2020-09-14,2020-09-15,1 days
8,123,178,199,10180.14350,9850.0,-40607.65050,-0.032430,2020-09-30,2020-10-30,30 days
9,116,203,270,10471.57900,11920.0,168016.83600,0.138319,2020-11-06,2021-02-16,102 days


**ドンチャンブレイクアウト**

In [29]:
import BacktestingStrategies.Strategy_Donchian as btstdnchn
importlib.reload(btstdnchn)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstdnchn.DnchnBreakout, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "dnchn_long": range(10, 50, 5),
    "dnchn_short": range(10, 50, 5),
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.dnchn_long > x.dnchn_short),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstdnchn.DnchnBreakout_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   58.245083
Equity Final [$]                1138303.99075
Equity Peak [$]                 1316531.12775
Return [%]                          13.830399
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    5.062548
Volatility (Ann.) [%]               14.817653
Sharpe Ratio                         0.341657
Sortino Ratio                         0.50271
Calmar Ratio                         0.356461
Max. Drawdown [%]                  -14.202257
Avg. Drawdown [%]                   -2.833261
Max. Drawdown Duration      209 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                   17
Win Rate [%]                        70.588235
Best Trade [%]                      10.073572
Worst Trade [%]                      -6.85641
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,93,27,34,10702.71750,9980.0,-67212.72750,-0.067527,2020-02-17,2020-02-27,10 days
1,108,65,107,8612.42150,9480.0,93698.47800,0.100736,2020-04-13,2020-06-16,64 days
2,103,129,166,9898.75750,10370.0,48537.97750,0.047606,2020-07-16,2020-09-10,56 days
3,100,186,194,10742.91550,10410.0,-33291.55000,-0.030989,2020-10-13,2020-10-23,10 days
4,95,210,230,10923.80650,10930.0,588.38250,0.000567,2020-11-17,2020-12-16,29 days
5,93,240,259,11094.64800,11250.0,14447.73600,0.014002,2020-12-30,2021-01-29,30 days
6,91,263,276,11597.12300,11930.0,30291.80700,0.028703,2021-02-04,2021-02-25,21 days
7,89,285,316,12119.69700,12900.0,69446.96700,0.064383,2021-03-10,2021-04-22,43 days
8,87,321,355,13184.94400,13340.0,13489.87200,0.011760,2021-04-30,2021-06-22,53 days
9,86,357,376,13566.82500,13630.0,5433.05000,0.004657,2021-06-24,2021-07-21,27 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                    61.72466
Equity Final [$]                  1251812.153
Equity Peak [$]                 1439737.25625
Return [%]                          25.181215
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    8.939627
Volatility (Ann.) [%]               15.890737
Sharpe Ratio                         0.562568
Sortino Ratio                         0.87742
Calmar Ratio                         0.651516
Max. Drawdown [%]                  -13.721261
Avg. Drawdown [%]                   -2.424041
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   18
Win Rate [%]                        72.222222
Best Trade [%]                      13.517511
Worst Trade [%]                     -6.752654
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,93,27,34,10702.71750,9980.0,-67212.72750,-0.067527,2020-02-17,2020-02-27,10 days
1,111,62,107,8351.13450,9480.0,125304.07050,0.135175,2020-04-08,2020-06-16,69 days
2,109,121,166,9687.71800,10370.0,74368.73800,0.070428,2020-07-06,2020-09-10,66 days
3,107,184,194,10562.02450,10410.0,-16266.62150,-0.014394,2020-10-09,2020-10-23,14 days
4,103,206,230,10742.91550,10930.0,19269.70350,0.017415,2020-11-11,2020-12-16,35 days
5,102,240,259,11094.64800,11250.0,15845.90400,0.014002,2020-12-30,2021-01-29,30 days
6,99,263,276,11597.12300,11930.0,32954.82300,0.028703,2021-02-04,2021-02-25,21 days
7,97,285,316,12119.69700,12900.0,75689.39100,0.064383,2021-03-10,2021-04-22,43 days
8,95,321,355,13184.94400,13340.0,14730.32000,0.011760,2021-04-30,2021-06-22,53 days
9,93,357,376,13566.82500,13630.0,5875.27500,0.004657,2021-06-24,2021-07-21,27 days


<Strategy DnchnBreakout(dnchn_long=15,dnchn_short=10)>

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   74.281392
Equity Final [$]                  1024570.513
Equity Peak [$]                  1405838.7425
Return [%]                           2.457051
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                      0.9297
Volatility (Ann.) [%]               19.423261
Sharpe Ratio                         0.047865
Sortino Ratio                         0.07276
Calmar Ratio                         0.030348
Max. Drawdown [%]                  -30.634255
Avg. Drawdown [%]                   -5.440452
Max. Drawdown Duration      747 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                                   29
Win Rate [%]                        62.068966
Best Trade [%]                       10.91167
Worst Trade [%]                    -12.314649
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,93,27,34,10702.71750,9980.0,-67212.72750,-0.067527,2020-02-17,2020-02-27,10 days
1,-100,35,36,9293.76700,9100.0,19376.70000,0.020849,2020-02-28,2020-03-02,3 days
2,-115,42,45,8239.01400,7340.0,103386.61000,0.109117,2020-03-10,2020-03-13,3 days
3,-130,46,51,8079.80600,7250.0,107874.78000,0.102701,2020-03-16,2020-03-24,8 days
4,135,65,107,8612.42150,9480.0,117123.09750,0.100736,2020-04-13,2020-06-16,64 days
5,129,129,166,9898.75750,10370.0,60790.28250,0.047606,2020-07-16,2020-09-10,56 days
6,-138,174,175,9701.73750,9840.0,-19080.22500,-0.014251,2020-09-24,2020-09-25,1 days
7,123,186,194,10742.91550,10410.0,-40948.60650,-0.030989,2020-10-13,2020-10-23,10 days
8,-130,198,199,9791.29200,9850.0,-7632.04000,-0.005996,2020-10-29,2020-10-30,1 days
9,-130,200,230,9731.58900,10930.0,-155793.43000,-0.123146,2020-11-02,2020-12-16,44 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   74.281392
Equity Final [$]                  1233185.378
Equity Peak [$]                   1481128.019
Return [%]                          23.318538
Buy & Hold Return [%]               62.750832
Return (Ann.) [%]                    8.318766
Volatility (Ann.) [%]               20.514719
Sharpe Ratio                         0.405502
Sortino Ratio                        0.672288
Calmar Ratio                         0.307479
Max. Drawdown [%]                  -27.054715
Avg. Drawdown [%]                   -4.032273
Max. Drawdown Duration      747 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                                   30
Win Rate [%]                        63.333333
Best Trade [%]                      13.517511
Worst Trade [%]                    -12.314649
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,93,27,34,10702.71750,9980.0,-67212.72750,-0.067527,2020-02-17,2020-02-27,10 days
1,-100,35,36,9293.76700,9100.0,19376.70000,0.020849,2020-02-28,2020-03-02,3 days
2,-115,42,45,8239.01400,7340.0,103386.61000,0.109117,2020-03-10,2020-03-13,3 days
3,-130,46,51,8079.80600,7250.0,107874.78000,0.102701,2020-03-16,2020-03-24,8 days
4,139,62,107,8351.13450,9480.0,156912.30450,0.135175,2020-04-08,2020-06-16,69 days
5,136,121,166,9687.71800,10370.0,92790.35200,0.070428,2020-07-06,2020-09-10,66 days
6,-145,174,175,9701.73750,9840.0,-20048.06250,-0.014251,2020-09-24,2020-09-25,1 days
7,131,184,194,10562.02450,10410.0,-19915.20950,-0.014394,2020-10-09,2020-10-23,14 days
8,-140,198,199,9791.29200,9850.0,-8219.12000,-0.005996,2020-10-29,2020-10-30,1 days
9,-140,200,230,9731.58900,10930.0,-167777.54000,-0.123146,2020-11-02,2020-12-16,44 days


<Strategy DnchnBreakout_WithShortPosition(dnchn_long=15,dnchn_short=10)>

In [30]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("OriginalStrategies.csv")